In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [3]:
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()))
url='https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(5)

#### First step: Tipo de proceso

In [5]:
#Select "Elecciones presidenciales" en "Tipo de proceso"
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()

#### Second step: Elecciones

In [6]:
# Select: Elección
driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()

# Get all the elections

## List of elections
elections_list = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')

## Elections
elections = [ i for i in elections_list.find_elements(By.TAG_NAME, "div") ]

# Dictionary of the elections and their drivers
dict_elections = { election.text : election for election in elections }

#### Dataframe final

In [7]:
df_resultados = pd.DataFrame(columns=['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS'])

#### Loop to get all the elections' tables

In [8]:

for x in list(dict_elections.keys())[1:]:
    elemento = dict_elections[x]
    elemento.click()

    # Select the election
    boton_ver_datos = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.ID, 'btnVerDatos'))
    ).click()
    time.sleep(5)
    
    # Go to the section "Candidatos y resultados"
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]').click()
    time.sleep(5)

    # Get the table for each election
    table_1= driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/table')
    table1_html=table_1.get_attribute("outerHTML")
    table1= pd.read_html(table1_html)[0]

    table1['ELECCIONES'] = x
    new_order1 = ['ELECCIONES', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']
    df1 = table1[new_order1]
    
    # Concat the new tables in the final dataframe
    df_resultados = pd.concat([df_resultados, df1], ignore_index=True)
    
    # Back to the beginning
    driver.back()
    time.sleep(5)

    driver.back()
    time.sleep(5)
    
    # On the list, to select a new election
    driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
    time.sleep(5)


C:\Users\kchauca\AppData\Local\Temp\ipykernel_27024\2334740008.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_27024\2334740008.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_27024\2334740008.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table1= pd.read_html(table1_html)[0]
C:\Users\kchauca\AppData\Local\Temp\ipykernel_27024\2334740008.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future

#### Results

In [9]:
# Verify the final output
df_resultados

,ELECCIONES,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2021 - 2DA VUELTA,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
1,PRESIDENCIAL 2021 - 2DA VUELTA,FUERZA POPULAR,8792117
2,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS EN BLANCO,121489
3,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS NULOS,1106816
4,PRESIDENCIAL 2021,PARTIDO POLITICO NACIONAL PERU LIBRE,2724752
...,...,...,...
152,PRESIDENCIAL 1936,PARTIDO REPUBLICANO,30803
153,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149
154,PRESIDENCIAL 1931,PARTIDO APRISTA PERUANO,106088
155,PRESIDENCIAL 1931,PARTIDO DESCENTRALISTA,21950


In [11]:
# Consider
file_name = "resultados_elecciones.xlsx"
sheet = "Resultados"

# Export the final df
df_resultados.to_excel(file_name, sheet_name=sheet, index=False)

print(f"All the results of presidential elections are in this file: {file_name}")

All the results of presidential elections are in this file: resultados_elecciones.xlsx
